In [ ]:
pip install office365

In [ ]:
pip install Office365-REST-Python-Client

In [ ]:
pip install psycopg2

In [ ]:
pip install sharepy

In [1]:
import pandas as pd
from pandas.api.types import infer_dtype
import psycopg2
from sqlalchemy import create_engine
from pandas.api.types import infer_dtype
import numpy as np
import requests
from openpyxl import load_workbook
import datetime as dt
from office365.runtime.auth.authentication_context import AuthenticationContext
from office365.sharepoint.client_context import ClientContext
from office365.sharepoint.files.file import File 
import io
from shareplum import Site
from shareplum import Office365
from shareplum.site import Version

In [2]:
URL = 'https://dynatrace-my.sharepoint.com'
url_shrpt = 'https://dynatrace-my.sharepoint.com/personal/rain_harrigan_dynatrace_com'
username_shrpt = 'rainy.li@dynatrace.com'
password_shrpt = 'Lovelry1314!'
folder_url_shrpt = '/personal/rain_harrigan_dynatrace_com/Documents/Desktop/BusSysBI/Access%20Replatform/manualadj/account/ready'
target_folder_url_shrpt = '/personal/rain_harrigan_dynatrace_com/Documents/Desktop/BusSysBI/Access%20Replatform/manualadj/account/succeeded'

In [3]:
ctx_auth = AuthenticationContext(url_shrpt)
if ctx_auth.acquire_token_for_user(username_shrpt, password_shrpt):
    ctx = ClientContext(url_shrpt, ctx_auth)
    web = ctx.web
    ctx.load(web)
    ctx.execute_query()
    print('Authenticated into sharepoint as: ',web.properties['Title'])

else:
    print(ctx_auth.get_last_error())

Authenticated into sharepoint as:  Harrigan, Rain


In [4]:
folder = ctx.web.get_folder_by_server_relative_url(folder_url_shrpt)
fold_names = []
sub_folders = folder.files #Replace files with folders for getting list of folders
ctx.load(sub_folders)
ctx.execute_query()

In [5]:
file_url_shrpt = '/personal/rain_harrigan_dynatrace_com/Documents/Desktop/BusSysBI/Access%20Replatform/manualadj/account/ready/data.csv'

In [6]:
response = File.open_binary(ctx, file_url_shrpt)
bytes_file_obj = io.BytesIO()
bytes_file_obj.write(response.content)
bytes_file_obj.seek(0)
df = pd.read_csv(bytes_file_obj)
df

,AccountNS_SK,Account Number,Account,Account Type,ProductType_FK,Product Type,Product Type Rollup1,Product Type Rollup2,Recurring,SaaS v Mgd
0,249,3501,Subscription - New,Income,5,SaaS - New,SaaS,SaaS,True,SaaS
1,17104,3501-01,Subscription - New : Subscription - Consumptio...,Income,12,SaaS - New,SaaS,SaaS,True,SaaS


In [7]:
host = "ec2-54-161-114-24.compute-1.amazonaws.com"
dbname = "d5lblgv4vjgptk"
port = 5432
user = "ua8hvj8ta2fqn7"
password = "p587b497a2f1d7bc7b136188eaa840ff3fc5b1180e5c6b47c3413f80915330eb4"
sslmode = "require"
postgres_str = f'postgresql://{user}:{password}@{host}:{port}/{dbname}'
conn = psycopg2.connect(postgres_str)
cnx = create_engine(postgres_str)
conn.autocommit = True
cur = conn.cursor()

In [8]:
cur.execute ( "truncate table datamart_arr.dimaccount_adjnewload" )
df.to_sql ( 'dimaccount_adjnewload', cnx, schema='datamart_arr', method='multi', if_exists='append',index=False,chunksize=5000)

In [9]:
cur.execute("CALL datamart_arr.sp_dimaccount_updatefrom_adjnewload();")
conn.commit()

In [10]:
target_file_url_shrpt = '/personal/rain_harrigan_dynatrace_com/Documents/Desktop/BusSysBI/Access%20Replatform/manualadj/account/succeeded/data_test_rainy.csv'

In [11]:
ctx_auth = AuthenticationContext(url_shrpt)
if ctx_auth.acquire_token_for_user(username_shrpt, password_shrpt):
    ctx = ClientContext(url_shrpt, ctx_auth)
    source_file = ctx.web.get_file_by_server_relative_url(file_url_shrpt)
    source_file.moveto(new_relative_url=target_file_url_shrpt, flag=1)
    ctx.execute_query()